In [41]:
import pandas as pd 
import time
import datetime
# from src.extract.trip_actions import get_target_date_components

In [42]:
raw_df = pd.read_csv('../data/raw/2025_11_13_raw.csv')
cleaned_df = pd.read_csv('../data/processed/2025_11_13_cleaned.csv')

In [43]:
raw_df

,company_name,bus_rating,seat_type,departure_time,pickup_point,arrival_date,arrival_time,dropoff_point,duration,price_original,...,departure_date,start_point,destination,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ
0,Hoàng Thuỷ,4.7 (3485),Limousine 34 Giường VIP,18:45,• Bến xe Miền Đông - Quầy vé 37,(14/11),05:45,• Bến Xe Đức Long Gia Lai,11h,350.000đ,...,"T5, 13/11/2025",Sài Gòn,Gia Lai,4.8,4.7,4.7,4.7,4.6,4.6,4.9
1,Sinh Diên Hồng,4.6 (237),Limousine 34 chỗ,17:30,• Bến xe An Sương,(14/11),04:15,• Bến Xe Đức Long Gia Lai,10h45m,370.000đ,...,"T5, 13/11/2025",Sài Gòn,Gia Lai,4.7,4.7,4.8,4.8,4.6,4.6,4.9
2,Phong Phú,4.5 (8677),Limousine 24 phòng đôi,20:00,• Văn phòng Hoàng Văn Thụ.,(14/11),07:45,• VP Gia Lai,11h45m,530.000đ,...,"T5, 13/11/2025",Sài Gòn,Gia Lai,4.5,4.5,4.6,4.5,4.4,4.4,4.7
3,Kính Diên Hồng,4.6 (1135),Limousine giường nằm 34 chỗ,18:40,• Bến Xe Miền Đông - Quầy vé 19,(14/11),05:40,• Bến Xe Đức Long Gia Lai,11h,350.000đ,...,"T5, 13/11/2025",Sài Gòn,Gia Lai,4.7,4.6,4.7,4.6,4.5,4.5,4.5
4,Đức Đạt,4.7 (1027),Limousine giường nằm 34 chỗ,20:10,• Bến xe Miền Đông - Quầy vé 17,(14/11),06:35,• Bến Xe Đức Long Gia Lai,10h25m,350.000đ,...,"T5, 13/11/2025",Sài Gòn,Gia Lai,4.6,4.5,4.6,4.5,4.5,4.5,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,VietNam Explore,4.4 (89),Ghế ngồi 16 chỗ,16:25,• Bảo tàng Điêu khắc Chăm Đà Nẵng,NaN,17:15,• Hoi An Ivy Hotel,0h50m,130.000đ,...,"T5, 13/11/2025",Đà Nẵng,Quảng Nam,4.6,4.6,4.4,4.6,4.6,4.4,5.0
168,Barri Ann Travel,4.7 (83),Ghế ngồi 16 chỗ (mới),11:30,• Bảo tàng Chăm,NaN,12:45,• Văn phòng Hội An,1h15m,130.000đ,...,"T5, 13/11/2025",Đà Nẵng,Quảng Nam,5.0,4.8,4.8,4.8,4.6,4.8,5.0
169,Barri Ann Travel,4.7 (83),Ghế ngồi 16 chỗ (mới),19:30,• Bảo tàng Chăm,NaN,20:45,• Văn phòng Hội An,1h15m,130.000đ,...,"T5, 13/11/2025",Đà Nẵng,Quảng Nam,5.0,4.8,4.8,4.8,4.6,4.8,5.0
170,Barri Ann Travel,4.7 (83),Ghế ngồi 16 chỗ (mới),14:30,• Bảo tàng Chăm,NaN,15:45,• Văn phòng Hội An,1h15m,130.000đ,...,"T5, 13/11/2025",Đà Nẵng,Quảng Nam,5.0,4.8,4.8,4.8,4.6,4.8,5.0


In [47]:
raw_df.isna().sum()

company_name               0
bus_rating                 0
seat_type                  0
departure_time             0
pickup_point               0
arrival_date              88
arrival_time               0
dropoff_point              0
duration                   0
price_original             0
price_discounted         106
percent_discount         106
departure_date             0
start_point                0
destination                0
An toàn                    0
Thông tin chính xác        0
Thông tin đầy đủ           0
Thái độ nhân viên          0
Tiện nghi & thoải mái      0
Chất lượng dịch vụ         0
Đúng giờ                  19
dtype: int64

In [45]:
cleaned_df

,company_name,departure_time,pickup_point,arrival_time,dropoff_point,price_original,price_discounted,departure_date,start_point,destination,...,rating_info_accuracy,rating_info_completeness,rating_staff_attitude,rating_comfort,rating_service_quality,rating_punctuality,rating_overall,reviewer_count,number_of_seat,duration_minutes
0,Hoàng Thuỷ,18:45:00,Bến xe,05:45,Bến xe,350000,300000,2025-11-13,Sài Gòn,Gia Lai,...,4.7,4.7,4.7,4.6,4.6,4.9,4.7,3485,34,660
1,Sinh Diên Hồng,17:30:00,Bến xe,04:15,Bến xe,370000,296000,2025-11-13,Sài Gòn,Gia Lai,...,4.7,4.8,4.8,4.6,4.6,4.9,4.6,237,34,645
2,Phong Phú,20:00:00,Văn phòng,07:45,Văn phòng,530000,399000,2025-11-13,Sài Gòn,Gia Lai,...,4.5,4.6,4.5,4.4,4.4,4.7,4.5,8677,24,705
3,Kính Diên Hồng,18:40:00,Bến xe,05:40,Bến xe,350000,0,2025-11-13,Sài Gòn,Gia Lai,...,4.6,4.7,4.6,4.5,4.5,4.5,4.6,1135,34,660
4,Đức Đạt,20:10:00,Bến xe,06:35,Bến xe,350000,300000,2025-11-13,Sài Gòn,Gia Lai,...,4.5,4.6,4.5,4.5,4.5,4.9,4.7,1027,34,625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,VietNam Explore,16:25:00,Other,17:15,Other,130000,120000,2025-11-13,Đà Nẵng,Quảng Nam,...,4.6,4.4,4.6,4.6,4.4,5.0,4.4,89,16,50
168,Barri Ann Travel,11:30:00,Other,12:45,Văn phòng,130000,0,2025-11-13,Đà Nẵng,Quảng Nam,...,4.8,4.8,4.8,4.6,4.8,5.0,4.7,83,16,75
169,Barri Ann Travel,19:30:00,Other,20:45,Văn phòng,130000,0,2025-11-13,Đà Nẵng,Quảng Nam,...,4.8,4.8,4.8,4.6,4.8,5.0,4.7,83,16,75
170,Barri Ann Travel,14:30:00,Other,15:45,Văn phòng,130000,0,2025-11-13,Đà Nẵng,Quảng Nam,...,4.8,4.8,4.8,4.6,4.8,5.0,4.7,83,16,75
